In [19]:
import sys, os, random

from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np
from pinecone import Pinecone

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Pinecone as langchain_pinecone

from dotenv import load_dotenv
load_dotenv("../.env")

True

In [13]:
import promptquality as pq
from promptquality import Scorers
from promptquality import SupportedModels

metrics = [
    Scorers.context_adherence,
    Scorers.latency,
    Scorers.pii,
    Scorers.toxicity,
    Scorers.tone,
    # Uncertainty, BLEU and ROUGE are automatically included
]

pq.login("console.demo.rungalileo.io")

galileo_handler = pq.GalileoPromptCallback(
    project_name="pinecone-webinar-30jan-test", scorers=metrics
)

Go to https://console.demo.rungalileo.io/get-token to generate a new API Key
👋 You have logged into 🔭 Galileo (https://console.demo.rungalileo.io/) as galileo@rungalileo.io.


In [14]:
df = pd.read_csv("../data/bigbasket_garnier.csv")
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,questions
0,103,Fructis Serum - Long & Strong,Beauty & Hygiene,Hair Care,Garnier,230.00,230.0,Shampoo & Conditioner,4.8,Garnier Fruits Long & Strong Strengthening Ser...,['What are the key ingredients in Fructis Seru...
1,163,Color Naturals Creme Riche Ultra Hair Color - ...,Beauty & Hygiene,Hair Care,Garnier,199.00,199.0,Hair Color,4.1,Garnier Color Naturals is creme hair colour wh...,['How long does Garnier Color Naturals Creme R...
2,213,Black Naturals Hair Colour Shade 1-Deep Black ...,Beauty & Hygiene,Hair Care,Garnier,108.78,117.0,Hair Color,4.2,It is an oil-enriched cream colour which gives...,['How long does Black Naturals Hair Colour las...
3,448,Black Naturals,Beauty & Hygiene,Hair Care,Garnier,37.00,39.0,Hair Color,4.2,Garnier Black Naturals is enriched with the go...,['How long does Garnier Black Naturals hair co...
4,1330,Skin Natural Serum sheetmask (Light complete +...,Beauty & Hygiene,Skin Care,Garnier,211.61,297.0,Face Care,NaN,Garnier introduces a new generation of face ma...,['Which Garnier face mask is enriched with Lem...


In [15]:
df["questions"] = df["questions"].apply(eval)
questions = df.explode("questions")["questions"].tolist()
random.Random(0).shuffle(questions)
# split questions into chunks of 5
questions = [questions[i:i + 5] for i in range(0, len(questions), 5)]
questions[0]

['Can Fructis Serum Long & Strong be used on both wet and dry hair?',
 'What is the main cause of hair breakage and split ends addressed by Garnier Fructis Long & Strong Strengthening Conditioner?',
 'Which product is the best for hydrating and brightening skin?',
 'How long does Color Naturals Creme Riche Ultra Hair Color last?',
 'What are the ingredients in Golden Brown Naturals Hair Colour?']

In [17]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
embeddings = OpenAIEmbeddings()

index = pc.Index("webinar")
vectorstore = langchain_pinecone(index, embeddings.embed_query, "text")
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/vectorstores.py#L553

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

/Users/pratikbhavsar/miniconda3/envs/rag/lib/python3.11/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [20]:
for q in tqdm(questions[0]):
    print("Question: ", q)
    print(qa.run(q, callbacks=[galileo_handler]))
    galileo_handler.finish()

  0%|          | 0/5 [00:00<?, ?it/s]

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [01:32<06:11, 92.96s/it]

🔭 View your prompt run on the Galileo console at: https://console.demo.rungalileo.io/prompt/chains/da5c707d-51fa-4002-87e3-5e17fbfbc66c/bd2ef39b-c32a-49fb-9454-c4a4f5e69226?taskType=12


 20%|██        | 1/5 [04:08<16:32, 248.09s/it]


KeyboardInterrupt: 